## Imports

In [1]:
!pip install pycm

    100% |████████████████████████████████| 61kB 2.6MB/s ta 0:00:011
    100% |████████████████████████████████| 512kB 10.9MB/s ta 0:00:01
    100% |████████████████████████████████| 215kB 36.7MB/s ta 0:00:01
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import boto3
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
from metrics                 import confusion_matrix_dataframe
from sklearn.model_selection import train_test_split
from sklearn.metrics         import balanced_accuracy_score, f1_score, confusion_matrix
from keras                   import regularizers
from keras.models            import Sequential
from keras.layers            import Dense, Flatten, Dropout
from keras.layers            import Conv2D, MaxPooling2D, BatchNormalization
from keras.utils             import np_utils
from keras.callbacks         import EarlyStopping, ModelCheckpoint
from IPython.core.display    import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%matplotlib inline

# Declaring a random state for the modeling process

np.random.seed(42)

Using TensorFlow backend.


## Table Contents
     
 
- [Reading In The New Data](#Reading-In-The-New-Data)
    - [Data Overview](#Data-Overview)
    - [Mapping Labels](#Mapping-Labels)
    
    
- [Preprocessing](#Preprocessing)
    - [Declaring X and y](#Declaring-X-and-y)    
    - [Train-Test Splitting The Data](#Train-Test-Splitting-The-Data)
    - [Reshaping The Data](#Reshaping-The-Data)
    
    
- [Modeling](#Modeling)
    - [Setting Up The Neural Network](#Setting-Up-The-Neural-Network)
    - [Compiling The Neural Network](#Compiling-The-Neural-Network)
    - [Fitting The Training Data](#Fitting-The-Training-Data)
    - [Generating Predictions](#Generating-Predictions)
 
 
- [Evaluation](#Evaluation)

## Reading In The Data

In [3]:
# Reading in my credentials

with open("access_key", "r") as key:
    access_key = key.read()
    
with open("secret_key", "r") as secret:
    secret_access_key = secret.read()
    
# Declaring the client as an s3 bucket
    
client = boto3.client("s3", 
                      aws_access_key_id     = access_key,
                      aws_secret_access_key = secret_access_key)

# Naming the bucket and file

bucket_name = "google-quick-draw"
object_key  = "animals.csv"

# Creating a csv object

csv_obj = client.get_object(Bucket = bucket_name, 
                            Key    = object_key)

body = csv_obj["Body"]

# Reading in the data

animals = pd.read_csv(body)

### Data Overview

In [4]:
# Checking the dimensions of the dataframe

animals.shape

(410123, 785)

In [5]:
# Looking at the head five rows

animals.head()

,label,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,bear,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
animals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410123 entries, 0 to 410122
Columns: 785 entries, label to 783
dtypes: int64(784), object(1)
memory usage: 2.4+ GB


In [7]:
# Looking for null values

animals.isnull().sum().sum()

0

### Mapping Labels

In [8]:
animals["label"].replace(to_replace = {"bear": 0,
                                       "cat" : 1,
                                       "dog" : 2},
                         inplace    = True)

# Making sure the labels are integers

animals["label"].dtype

dtype('int64')

## Preprocessing

### Declaring X and y

In [9]:
# X variable

X = np.array(animals.drop(columns = "label")) 

# y variable

y = animals["label"].values

### Train-Test Splitting The Data

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state = 42, 
                                                    test_size    = 0.3)

# Keeping copies of the original labels

y_train_org = y_train

y_test_org  = y_test

In [11]:
# Scaling each X value

X_train = X_train.astype(float)
X_train /= 255

X_test  = X_test.astype(float)
X_test  /= 255

# Making sure the y variables are categorical

y_train = np_utils.to_categorical(y_train, 3)

y_test  = np_utils.to_categorical(y_test, 3)

### Reshaping The Data

In [12]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)

X_test  = X_test.reshape(X_test.shape[0], 28, 28, 1)

# Checking the data type of the train and test sets

print(f"X_train's data type is {type(X_train)}")

print(f"X_test's data type is {type(X_test)}")

X_train's data type is <class 'numpy.ndarray'>
X_test's data type is <class 'numpy.ndarray'>


## Modeling

### Setting Up The Neural Network

#### Instantiating The Model

In [32]:
cnn = Sequential()

#### Adding Layers

In [33]:
# Convolutional layers

# The first convolutional layer

cnn.add(Conv2D(filters     = 6,
               kernel_size = 3,
               activation  = "relu",
               input_shape = (28,28,1)))

# The first maxpooling layer

cnn.add(MaxPooling2D(pool_size = (2,2)))

# The second convolutional layer

cnn.add(Conv2D(128,
               kernel_size = 3,
               activation  = "relu"))

# The second maxpooling layer

cnn.add(MaxPooling2D(pool_size = (2,2)))


In [34]:
# Dense layers

# Flattening the data

cnn.add(Flatten())

# Adding the first dense layer

cnn.add(Dense(128,
              activation         = "relu",
              kernel_regularizer = regularizers.l2(0.001)))

# Adding batch normalization

cnn.add(BatchNormalization())

# Adding the first dropout

cnn.add(Dropout(rate = 0.2))

# Adding the second dense layer

cnn.add(Dense(64,
              activation         = "relu",
              kernel_regularizer = regularizers.l2(0.001)))

# Adding batch normalization

cnn.add(BatchNormalization())
                
# Adding second dropout

cnn.add(Dropout(rate = 0.2))

# Adding the ouput layer

cnn.add(Dense(3,
              activation = "softmax"))

### Compiling The Neural Network

In [35]:
cnn.compile(loss      = "categorical_crossentropy",
            optimizer = "adam",
            metrics   = ["acc"])

### Fitting The Training Data

In [36]:
callback = [EarlyStopping(monitor  = 'val_loss', 
                          patience = 1,
                          verbose  = 0)]

In [37]:
cnn.fit(X_train,
        y_train,
        callbacks       = callback,
        validation_data = (X_train, y_train),
        batch_size      = 256,
        epochs          = 25,
        verbose         = 0)

In [38]:
# Training predictions

train_preds = cnn.predict_classes(X_train,
                                  batch_size = 100,
                                  verbose    = 0).ravel()

# Test predictions

test_preds = cnn.predict_classes(X_test,
                                 batch_size = 100,
                                 verbose    = 0).ravel()

In [39]:
print(f"The training balanced accuracy is: {balanced_accuracy_score(y_train_org, train_preds)}")
print(f'The training recall score is     : {f1_score(y_train_org, train_preds, average = "macro")}')

The training balanced accuracy is: 0.8505920368991462
The training recall score is     : 0.8509005250885253


In [40]:
print(f"The test balanced accuracy is: {balanced_accuracy_score(y_test_org, test_preds)}")
print(f'The test recall score is     : {f1_score(y_test_org, test_preds, average = "macro")}')

The test balanced accuracy is: 0.8419759522386387
The test recall score is     : 0.8425191048875815


In [41]:
confusion_matrix_dataframe(y_train_org,
                           train_preds,
                           columns = ["Bear", "Cat", "Dog"],
                           index   = ["Bear", "Cat", "Dog"])

,Bear,Cat,Dog
Bear,81337,4179,8889
Cat,2804,73768,9433
Dog,11123,6747,88806


[Top](#Table-Of-Contents)